In [ ]:
import sys

%load_ext autoreload
%load_ext line_profiler
%autoreload 2


sys.path.append("..")

In [ ]:
import csv
import os

import matplotlib.pyplot as plt
import numpy as np
import scienceplots  # noqa: F401
from joblib import Parallel, delayed
from matplotlib import rc
from tqdm.notebook import tqdm

from zfista.problems import FDS, JOS1, SD

In [ ]:
fig_path = os.path.abspath(os.path.join("./figs"))
data_path = os.path.abspath(os.path.join("./data"))
os.makedirs(fig_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)
rc("text", usetex=True)
plt.style.use(["science", "bright"])

In [ ]:
from fractions import Fraction

nesterov_ratios_f = [
    (0, 0),
    (0, Fraction(1, 8)),
    (0, Fraction(1, 4)),
    (Fraction(1, 6), Fraction(1, 144)),
    (Fraction(1, 6), Fraction(37, 288)),
    (Fraction(1, 6), Fraction(1, 4)),
    (Fraction(1, 4), Fraction(1, 64)),
    (Fraction(1, 4), Fraction(17, 128)),
    (Fraction(1, 4), Fraction(1, 4)),
    (Fraction(1, 2), Fraction(1, 16)),
    (Fraction(1, 2), Fraction(5, 32)),
    (Fraction(1, 2), Fraction(1, 4)),
    (Fraction(3, 4), Fraction(9, 64)),
    (Fraction(3, 4), Fraction(25, 128)),
    (Fraction(3, 4), Fraction(1, 4)),
]
nesterov_ratios = [tuple(map(float, t)) for t in nesterov_ratios_f]


def generate_start_points(low, high, n_features, n_samples=1000):
    return [
        np.random.uniform(low=low, high=high, size=n_features) for _ in range(n_samples)
    ]


def run(
    problem,
    start_points,
    tol=1e-5,
    nesterov=False,
    nesterov_ratio=(0, 0.25),
    n_jobs=-1,
    verbose=False,
):
    results = Parallel(n_jobs=n_jobs, verbose=10)(
        delayed(problem.minimize_proximal_gradient)(
            x0,
            tol=tol,
            nesterov=nesterov,
            nesterov_ratio=nesterov_ratio,
            return_all=False,
            verbose=verbose,
        )
        for x0 in start_points
    )
    return results


def show_Pareto_front(
    problem, results, s=15, alpha=0.75, fname=None, elev=15, azim=130, linewidths=0.1
):
    fig = plt.figure(figsize=(7.5, 12.5), dpi=100)
    if problem.n_objectives == 2:
        axs = [fig.add_subplot(5, 3, i + 1) for i in range(15)]
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    if problem.n_objectives == 3:
        axs = [
            fig.add_subplot(5, 3, i + 1, projection="3d", clip_on=True)
            for i in range(15)
        ]
        for i in range(15):
            axs[i].view_init(elev=elev, azim=azim)
        fig.subplots_adjust(left=0, right=1, bottom=0, top=0.6)
    for i, result in tqdm(enumerate(results)):
        ax = axs[i]
        ax.set_title("$(a, b) = (" + ",".join(map(str, nesterov_ratios_f[i])) + ")$")
        for result_k in tqdm(result):
            F_pareto = result_k.fun
            ax.scatter(
                *F_pareto,
                color="#2980b9",
                marker=".",
                s=s,
                alpha=alpha,
                linewidths=linewidths,
            )
        ax.set_xlabel(r"$F_1$", fontsize=10)
        ax.set_ylabel(r"$F_2$", fontsize=10)
        ax.tick_params(labelsize=8)
        if problem.n_objectives == 3:
            ax.set_zlabel(r"$F_3$", fontsize=10)
    fig.tight_layout()
    if fname is not None:
        plt.savefig(fig_path + "/" + fname, bbox_inches="tight")


def get_stats(results):
    nits = [result.nit for result in results]
    times = [result.time for result in results]
    stats = {
        "nit": {"mean": np.mean(nits), "std": np.std(nits), "max": np.max(nits)},
        "time": {
            "mean": np.mean(times),
            "std": np.std(times),
            "max": np.max(times),
        },
    }
    return stats

## JOS1
Minimize
$$
f_1(x) = \frac{1}{n} \| x \|_2^2, \quad f_2(x) = \frac{1}{n} \| x - 2\|_2^2
$$
subject to $x \in \mathbf{R^n}$.

In [ ]:
n_features = 50
problem_JOS1 = JOS1(n_features=n_features)
start_points_JOS1 = generate_start_points(low=-2, high=4, n_features=n_features)

In [ ]:
results_JOS1 = [
    run(problem_JOS1, start_points_JOS1, nesterov=True, nesterov_ratio=nesterov_ratio)
    for nesterov_ratio in tqdm(nesterov_ratios)
]

### Complexity

In [ ]:
import pprint

stats_JOS1 = {
    ",".join(map(str, nesterov_ratios_f[i])): get_stats(results_JOS1[i])
    for i in range(len(nesterov_ratios))
}
pprint.pprint(stats_JOS1)

with open(data_path + "/JOS1_ab.csv", "w") as f:
    writer = csv.writer(f, escapechar=" ", quoting=csv.QUOTE_NONE)
    for k, v in stats_JOS1.items():
        writer.writerow([k, round(v["time"]["mean"], 3), round(v["nit"]["mean"], 3)])

In [ ]:
show_Pareto_front(problem_JOS1, results_JOS1, fname="JOS1_ab.pdf")

## JOS1 + $\ell_1$ penalty
Minimize
$$
F_1(x) = \frac{1}{n} \| x \|_2^2 + \frac{1}{n} \|x\|_1, \quad F_2(x) = \frac{1}{n} \| x - 2\|_2^2 + \frac{1}{2n} \|x - 1\|_1
$$
subject to $x \in \mathbf{R}^n$.

In [ ]:
n_features = 50
problem_JOS1_L1 = JOS1(
    n_features=n_features,
    l1_ratios=(1 / n_features, 1 / n_features / 2),
    l1_shifts=(0, 1),
)
start_points_JOS1_L1 = generate_start_points(low=-2, high=4, n_features=n_features)

In [ ]:
results_JOS1_L1 = [
    run(
        problem_JOS1_L1,
        start_points_JOS1_L1,
        nesterov=True,
        nesterov_ratio=nesterov_ratio,
    )
    for nesterov_ratio in tqdm(nesterov_ratios)
]

### Complexity

In [ ]:
stats_JOS1_L1 = {
    ",".join(map(str, nesterov_ratios_f[i])): get_stats(results_JOS1_L1[i])
    for i in range(len(nesterov_ratios))
}
pprint.pprint(stats_JOS1_L1)

with open(data_path + "/JOS1_L1_ab.csv", "w") as f:
    writer = csv.writer(f, escapechar=" ", quoting=csv.QUOTE_NONE)
    for k, v in stats_JOS1_L1.items():
        writer.writerow([k, round(v["time"]["mean"], 3), round(v["nit"]["mean"], 3)])

In [ ]:
show_Pareto_front(problem_JOS1_L1, results_JOS1_L1, fname="JOS1_L1_ab.pdf")

## SD
Minimize
$$F_1(x) = 2 x_1 + \sqrt{2} x_2 + \sqrt{2} x_3 + x_4, \quad F_2(x) = \frac{2}{x_1} + \frac{2 \sqrt{2}}{x_2} + \frac{2 \sqrt{2}}{x_3} + \frac{2}{x_4}$$
subject to $(1, \sqrt{2}, \sqrt{2}, 1)^\top \le x \le (3, 3, 3, 3)^\top$.

In [ ]:
problem_SD = SD()
start_points_SD = generate_start_points(
    low=[1, np.sqrt(2), np.sqrt(2), 1],
    high=[3, 3, 3, 3],
    n_features=problem_SD.n_features,
)

In [ ]:
results_SD = [
    run(problem_SD, start_points_SD, nesterov=True, nesterov_ratio=nesterov_ratio)
    for nesterov_ratio in tqdm(nesterov_ratios)
]

### Complexity

In [ ]:
stats_SD = {
    ",".join(map(str, nesterov_ratios_f[i])): get_stats(results_SD[i])
    for i in range(len(nesterov_ratios))
}
pprint.pprint(stats_SD)

with open(data_path + "/SD_ab.csv", "w") as f:
    writer = csv.writer(f, escapechar=" ", quoting=csv.QUOTE_NONE)
    for k, v in stats_SD.items():
        writer.writerow([k, round(v["time"]["mean"], 3), round(v["nit"]["mean"], 3)])

In [ ]:
%matplotlib inline
show_Pareto_front(problem_SD, results_SD, fname="SD_ab.pdf")

## FDS
Minimize
$$F_1(x) = \frac{1}{n^2} \sum_{i = 1}^n i (x_i - i)^4, \quad F_2(x) = \exp \left( \sum_{i = 1}^n \frac{x_i}{n} \right) + \|x\|_2^2, \quad F_3(x) = \frac{1}{n(n + 1)} \sum_{i = 1}^n i (n - i + 1) \exp (- x_i)$$
subject to $x \in \mathbf{R}^n$.

In [ ]:
n_features = 10
problem_FDS = FDS(n_features=n_features)
start_points_FDS = generate_start_points(low=-2, high=2, n_features=n_features)

In [ ]:
results_FDS = [
    run(problem_FDS, start_points_FDS, nesterov=True, nesterov_ratio=nesterov_ratio)
    for nesterov_ratio in tqdm(nesterov_ratios)
]

### Complexity

In [ ]:
stats_FDS = {
    ",".join(map(str, nesterov_ratios_f[i])): get_stats(results_FDS[i])
    for i in range(len(nesterov_ratios))
}
pprint.pprint(stats_FDS)

with open(data_path + "/FDS_ab.csv", "w") as f:
    writer = csv.writer(f, escapechar=" ", quoting=csv.QUOTE_NONE)
    for k, v in stats_FDS.items():
        writer.writerow([k, round(v["time"]["mean"], 3), round(v["nit"]["mean"], 3)])

In [ ]:
%matplotlib inline
show_Pareto_front(problem_FDS, results_FDS, fname="FDS_ab.pdf")

## FDS CONSTRAINED
Minimize
$$F_1(x) = \frac{1}{n^2} \sum_{i = 1}^n i (x_i - i)^4, \quad F_2(x) = \exp \left( \sum_{i = 1}^n \frac{x_i}{n} \right) + \|x\|_2^2, \quad F_3(x) = \frac{1}{n(n + 1)} \sum_{i = 1}^n i (n - i + 1) \exp (- x_i)$$
subject to $x \in \mathbf{R}_+^n$.

In [ ]:
n_features = 10
problem_FDS_CONSTRAINED = FDS(n_features=n_features, bounds=(0, np.inf))
start_points_FDS_CONSTRAINED = generate_start_points(
    low=0, high=2, n_features=n_features
)

In [ ]:
results_FDS_CONSTRAINED = [
    run(
        problem_FDS_CONSTRAINED,
        start_points_FDS_CONSTRAINED,
        nesterov=True,
        nesterov_ratio=nesterov_ratio,
    )
    for nesterov_ratio in tqdm(nesterov_ratios)
]

In [ ]:
stats_FDS_CONSTRAINED = {
    ",".join(map(str, nesterov_ratios_f[i])): get_stats(results_FDS_CONSTRAINED[i])
    for i in range(len(nesterov_ratios))
}
pprint.pprint(stats_FDS_CONSTRAINED)

with open(data_path + "/FDS_CONSTRAINED_ab.csv", "w") as f:
    writer = csv.writer(f, escapechar=" ", quoting=csv.QUOTE_NONE)
    for k, v in stats_FDS_CONSTRAINED.items():
        writer.writerow([k, round(v["time"]["mean"], 3), round(v["nit"]["mean"], 3)])

In [ ]:
%matplotlib inline
show_Pareto_front(
    problem_FDS_CONSTRAINED, results_FDS_CONSTRAINED, fname="FDS_CONSTRAINED_ab.pdf"
)